In [51]:
import pandas as pd
import polars as pl
from timeit import timeit
import os, sys
os.chdir('/gpfs/data/healthcare-allocate/CLIF-MIMIC/code')

proj_root = "/gpfs/data/healthcare-allocate/CLIF-MIMIC"
if proj_root not in sys.path:
    sys.path.append(proj_root)

from custom_utils import load_gz_csv

ImportError: cannot import name 'lookup_item' from 'custom_utils' (/gpfs/data/healthcare-allocate/CLIF-MIMIC/code/custom_utils.py)

In [50]:
print(os.getcwd())

with open("/gpfs/data/healthcare-allocate/CLIF-MIMIC/code/custom_utils.py") as f:
    print(f.read())

/gpfs/data/healthcare-allocate/CLIF-MIMIC/code
import pandas as pd
import gzip

def load_gz_csv(file_path):
    with gzip.open(file_path, 'rt', encoding = 'utf-8') as f:
        df = pd.read_csv(f)
    return df

def lookup_item(df, kw: str, col: str = "label", case: bool = False):
    '''
    col = {"label", "abbr"}
    '''
    if col == "abbr": col == "abbreviation"
    output = df[
        df[col].str.contains(kw, case = case, na = False)
    ]
    
    return output


In [41]:
def lookup_item(kw: str, col: str = "label", case: bool = False, df = d_items):
    '''
    col = {"label", "abbr"}
    '''
    if col == "abbr": col == "abbreviation"
    return df[
        df[col].str.contains(kw, case = case, na = False)
    ]

## hosp

In [2]:
d_labitems = pd.read_csv("mimic-iv-2.2/hosp/d_labitems.csv.gz")

In [4]:
# d_labitems.head(10)
contains = d_labitems["label"].str.contains("temp", case=False, na=False)
d_labitems[contains]

,itemid,label,fluid,category
23,50825,Temperature,Blood,Blood Gas


In [19]:
d_labitems["fluid"].unique()
d_labitems["category"].unique()

array(['Blood Gas', 'Chemistry', 'Hematology'], dtype=object)

In [12]:
omr = pd.read_csv("../mimic-iv-2.2/hosp/omr.csv.gz")
omr.head(10)

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15
5,10000032,2180-05-07,2,Weight (Lbs),92.15
6,10000032,2180-05-07,3,Weight (Lbs),92.15
7,10000032,2180-05-07,4,Weight (Lbs),92.15
8,10000032,2180-05-07,5,Weight (Lbs),92.15
9,10000032,2180-05-07,6,Weight (Lbs),92.15


## `icu`

### `d_items`

In [8]:
d_items = pd.read_csv("mimic-iv-2.2/icu/d_items.csv.gz")

In [9]:
# d_items.head()
d_items[
    d_items["label"].str.contains("Temperature", case=False, na=False)
]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
337,223761,Temperature Fahrenheit,Temperature F,chartevents,Routine Vital Signs,°F,Numeric,NaN,NaN
338,223762,Temperature Celsius,Temperature C,chartevents,Routine Vital Signs,°C,Numeric,NaN,NaN
505,224027,Skin Temperature,Skin Temp,chartevents,Skin - Assessment,NaN,Text,NaN,NaN
767,224642,Temperature Site,Temp Site,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN
790,224674,Changes in Temperature,Changes in Temperature,chartevents,Toxicology,NaN,Text,NaN,NaN
1814,226329,Blood Temperature CCO (C),Blood Temp CCO (C),chartevents,Routine Vital Signs,°C,Numeric,NaN,NaN
2097,227054,TemperatureF_ApacheIV,TemperatureF_ApacheIV,chartevents,Scores - APACHE IV (2),°F,Numeric,NaN,NaN
2776,228242,Pt. Temperature (BG) (SOFT),Pt. Temperature (BG) (SOFT),chartevents,Labs,NaN,Numeric,NaN,NaN
3466,229236,Cerebral Temperature (C),Cerebral T (C),chartevents,Hemodynamics,°C,Numeric,NaN,NaN


### `chartevents`

In [4]:
t1 = timeit(
    lambda: pd.read_csv("mimic-iv-2.2/icu/chartevents.csv.gz", nrows=100000), number = 10
)

t2 = timeit(
    lambda: pl.read_csv("mimic-iv-2.2/icu/chartevents.csv.gz", n_rows=100000), number = 10
)

print(t1,t2)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
1.3999643280403689 1.177795070107095


In [3]:
# resave into parquet
chartevents = load_gz_csv("mimic-iv-2.2/icu/chartevents.csv.gz")
chartevents.to_parquet('mimic-iv-2.2/icu/chartevents.parquet')  

In [3]:
chartevents = pd.read_parquet('mimic-iv-2.2/icu/chartevents.parquet')

In [4]:
chartevents.head(20)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82,82.0,mmHg,0.0
1,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220180,59,59.0,mmHg,0.0
2,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220181,63,63.0,mmHg,0.0
3,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220045,94,94.0,bpm,0.0
4,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85,85.0,mmHg,0.0
5,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220180,55,55.0,mmHg,0.0
6,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220181,62,62.0,mmHg,0.0
7,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220210,20,20.0,insp/min,0.0
8,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220277,95,95.0,%,0.0
9,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220045,97,97.0,bpm,0.0
